In [1]:
import math
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import numpy as np

In [2]:
print(torch.cuda.is_available())

True


In [3]:
#论文参数全局变量定义区
maxBPEpoch=1
maxCCIter=1
maxEAGen=10
tao=0.005
eta=0.1
alpha=0.3
beta=10e-5
betas=10e-3
sigmas=5e-4
HiddenLayer1=300
HiddenLayer2=100
OutputLayer=10
R=HiddenLayer1+HiddenLayer2+OutputLayer
#Q=int(0.3*R)
Q=1
NP=100
LP=2
szd=10000
szds=int(0.2*szd)
imgSize=784
miniBatch=100
maturityBatch=szds

In [4]:
class fnn(nn.Module):
	def __init__(self):
		super(fnn, self).__init__()
		self.hidden1 = nn.Sequential(nn.Linear(imgSize, HiddenLayer1),nn.Sigmoid())
		self.hidden2 = nn.Sequential(nn.Linear(HiddenLayer1, HiddenLayer2),nn.Sigmoid())
		self.output  = nn.Sequential(nn.Linear(HiddenLayer2, OutputLayer),nn.Sigmoid())

	def forward(self, x):
		x = self.get_hidden2_output(x)
		o = self.output (x)
		return o

	def get_hidden1_output(self,x):
		return self.hidden1(x)
	
	def get_hidden2_output(self,x):
		x=self.hidden1(x)
		return self.hidden2(x)

net=fnn()
net.parameters

criterion = nn.MSELoss()#不如用交叉熵好，MSE输出太平均了
optimizer = torch.optim.SGD(net.parameters(), lr=0.1,weight_decay=beta)

Layers_amount=len(list(net.children()))

In [5]:
import torchvision.datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,sampler

train_dataset=torchvision.datasets.MNIST(root='./data/mnist',train=True,transform=transforms.ToTensor(),download=True)
print(train_dataset)
test_dataset=torchvision.datasets.MNIST(root='./data/mnist',train=False,transform=transforms.ToTensor(),download=True)


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/mnist
    Split: Train
    StandardTransform
Transform: ToTensor()


In [6]:
#初始化
for _,param in enumerate(net.state_dict().values()):
	if param.dim()==2 :
		print("weight",param.size())
		param=torch.zeros(param.size())+2*math.sqrt(6)*(random.uniform(0,1)-0.5)/math.sqrt(param.size(0)+param.size(1))
	elif param.dim()==1 :
		print("bias",param.size())
		param=torch.zeros(param.size())+2*(random.uniform(0,1)-0.5)
net.hidden1[0].weight[0].size()

weight torch.Size([300, 784])
bias torch.Size([300])
weight torch.Size([100, 300])
bias torch.Size([100])
weight torch.Size([10, 100])
bias torch.Size([10])


torch.Size([784])

In [7]:
def Maturity_Calc_p(index,p,img):
    maturity=0
    if index<HiddenLayer1: 
        temp=torch.reshape(p[0:imgSize],(imgSize,1))
        oi=torch.sigmoid(torch.mm(img,temp)+p[-1])
    elif index<HiddenLayer1+HiddenLayer2:
        x=net.get_hidden1_output(img)
        temp=torch.reshape(p[0:HiddenLayer1],(HiddenLayer1,1))
        oi=torch.sigmoid(torch.mm(x,temp)+p[-1])
    else:
        x=net.get_hidden2_output(img)
        temp=torch.reshape(p[0:HiddenLayer2],(HiddenLayer2,1))
        oi=torch.sigmoid(torch.mm(x,temp)+p[-1])
    for o in oi:
        if o<alpha or o>1-alpha: maturity+=1
    return maturity

In [8]:
def Maturity_Calc(i,Pcc,img):
    maturity=0
    if i<HiddenLayer1: 
        temp=torch.reshape(Pcc[i][0:imgSize],(imgSize,1))
        oi=torch.sigmoid(torch.mm(img,temp)+Pcc[i][-1])
    elif i<HiddenLayer1+HiddenLayer2:
        x=net.get_hidden1_output(img)
        temp=torch.reshape(Pcc[i][0:HiddenLayer1],(HiddenLayer1,1))
        oi=torch.sigmoid(torch.mm(x,temp)+Pcc[i][-1])
    else:
        x=net.get_hidden2_output(img)
        temp=torch.reshape(Pcc[i][0:HiddenLayer2],(HiddenLayer2,1))
        oi=torch.sigmoid(torch.mm(x,temp)+Pcc[i][-1])
    for o in oi:
        if o<alpha or o>1-alpha: maturity+=1
    print(i,maturity)
    return maturity

In [9]:
def Get_Maturity(Pcc,Dindices):
    print("Getting Maturity of all subtasks, listed below:")
    sub=[]
    i=0
    Ds_Sampler=Ds_Sample(Dindices)
    while i<10:
        Ds_loader=DataLoader(dataset=train_dataset,sampler=Ds_Sampler,batch_size=szds)
        #求成熟度怎么设batch不知道
        for img,_ in Ds_loader:
            img=img.view(maturityBatch, -1)
            sub.append(Maturity_Calc(i,Pcc,img))
            i+=1
            if i>=R: break
    return sub

In [10]:
def Neural_Decompose(Pcc):
    for i in range(R):
        Pcc.append(torch.Tensor()) 
    flag_w=0
    flag_b=0
    for param in net.state_dict().values():
        if param.dim()==2 :
            for i in range(param.size(0)):
                Pcc[i+flag_w]=torch.cat((Pcc[i+flag_w],param[i]),0)
            flag_w+=param.size(0)
        elif param.dim()==1 :
            for i in range(param.size(0)):
                Pcc[i+flag_b]=torch.cat((Pcc[i+flag_b],torch.Tensor([param[i]])),0)
            flag_b+=param.size(0)

In [11]:
def Maturity_Rank(sub):
    return [index for index,_ in sorted(list(enumerate(sub)),
    key=lambda shenmewanyi: shenmewanyi[1],reverse=True)]

In [12]:
def Ds_Sample(Dindices):
    random.shuffle(Dindices)
    #每次都shuffle整个数据集大小的indices，占用时间
    return sampler.SubsetRandomSampler(Dindices)

In [13]:
def Spawn_Group(Psub):
    Psub_ip=Psub.repeat(NP-1,1)
    multiplier=torch.rand(NP-1,1)*2
    Psub_ip*=multiplier
    Psub_ip=torch.cat((Psub_ip,Psub.reshape(1,Psub.size(0))),dim=0)
    return Psub_ip

In [14]:
def Parameter_Load(chrome,index):
    if index<HiddenLayer1:
        net.hidden1[0].weight.requires_grad=False
        net.hidden1[0].bias.requires_grad=False

        net.hidden1[0].weight[index]=chrome[:imgSize]
        net.hidden1[0].bias[index]=chrome[-1]

        net.hidden1[0].weight.requires_grad=True
        net.hidden1[0].bias.requires_grad=True
    elif index<HiddenLayer1+HiddenLayer2:
        net.hidden2[0].weight.requires_grad=False
        net.hidden2[0].bias.requires_grad=False

        net.hidden2[0].weight[index-HiddenLayer1]=chrome[:HiddenLayer1]
        net.hidden2[0].bias[index-HiddenLayer1]=chrome[-1]

        net.hidden2[0].weight.requires_grad=True
        net.hidden2[0].bias.requires_grad=True
    else:
        net.output[0].weight.requires_grad=False
        net.output[0].bias.requires_grad=False

        net.output[0].weight[index-HiddenLayer1-HiddenLayer2]=chrome[:HiddenLayer2]
        net.output[0].bias[index-HiddenLayer1-HiddenLayer2]=chrome[:HiddenLayer2]

        net.output[0].weight.requires_grad=True
        net.output[0].bias.requires_grad=True

In [15]:
def Group_Evaluate(Fs,index,Psub_group,img,label_onehot):
    #将每一个体的局部参数装载给网络
    losses=[]
    for p in Psub_group:
        Parameter_Load(p,index)
        out=net(img)
        losses.append(criterion(out,label_onehot))
    #个体评价=个体输出+个体成熟度
    maturity=[]
    for p in Psub_group:
        maturity.append(Maturity_Calc_p(index,p,img))
    for i in range(len(Psub_group)):
        Fs.append(1/(losses[i]+(10e-8))+sigmas*maturity[i])
    print("Individuals of Subtask",index,"Evaluated Fs")

In [16]:
class Strategy_Candidate():
    def __init__(self,Psub_group):
        self.p_rand_1_bin=0.25
        self.p_rand_to_best_2_bin=0.25
        self.p_rand_2_bin=0.25
        self.p_current_to_rand_1=0.25
        
        self.F=self.F_determine()
        self.K=self.K_determine()
        
        self.CRm=0.5
        self.CR=self.CR_determine(Psub_group)
        
    def CR_determine(self,Psub_group):
        CR=np.zeros([NP,Psub_group[0].size(0)])
        for i in range(NP):
            for j in range(Psub_group[0].size(0)):
                temp=random.normalvariate(self.CRm,0.1)
                while temp<0 or temp>1:
                    temp=random.normalvariate(self.CRm,0.1)
                CR[i][j]=temp
        return CR
    def K_determine(self):
        return random.uniform(0,1)
    def F_determine(self):
        F=[]
        for i in range(NP):
            F.append(random.normalvariate(0.5,0.3))
        return F
    
    def Action(self,i,Psub_group,Fs):
        roulette=random.uniform(0,1)
        if roulette<self.p_rand_1_bin:
            print("rand_1_bin executing on individual",i)
            return self.rand_1_bin(i,Psub_group)
        elif roulette<self.p_rand_1_bin+self.p_rand_to_best_2_bin:
            print("rand_to_best_2_bin executing on individual",i)
            return self.rand_to_best_2_bin(i,Psub_group,Fs)
        elif roulette<self.p_rand_1_bin+self.p_rand_to_best_2_bin+self.p_rand_2_bin:
            print("rand_2_bin executing on individual",i)
            return self.rand_2_bin(i,Psub_group)
        else: 
            print("current_to_rand_1 executing on individual",i)
            return self.current_to_rand_1(i,Psub_group)
        
    def rand_1_bin(self,i,Psub_group):
        jrand=random.randint(0,Psub_group[0].size(0))
        Ui=torch.Tensor()
        for j in range(Psub_group[0].size(0)):
            if random.uniform(0,1)<self.CR[i][j] or j==jrand:
                r=[] 
                r_toPick=list(range(NP))
                for pickr in range(3):
                    temp=random.choice(r_toPick)
                    r.append(temp)
                    r_toPick.remove(temp)
                cat=torch.Tensor([Psub_group[r[0]][j]+self.F[i]*(Psub_group[r[1]][j]-Psub_group[r[2]][j])])
                Ui=torch.cat((Ui,cat),dim=0)
            else: Ui=torch.cat((Ui,torch.Tensor([Psub_group[i][j]])),dim=0)
        return Ui
    
    def rand_to_best_2_bin(self,i,Psub_group,Fs):
        
        best=Fs.index(max(Fs))

        jrand=random.randint(0,Psub_group[0].size(0))
        Ui=torch.Tensor()
        for j in range(Psub_group[0].size(0)):
            if random.uniform(0,1)<self.CR[i][j] or j==jrand:
                r=[]
                r_toPick=list(range(NP))
                for pickr in range(4):
                    temp=random.choice(r_toPick)
                    r.append(temp)
                    r_toPick.remove(temp)
                cat=torch.Tensor(
                    [Psub_group[i][j]+self.F[i]*(Psub_group[best][j]-Psub_group[i][j])
                                    +self.F[i]*(Psub_group[r[0]][j]-Psub_group[r[1]][j])
                                    +self.F[i]*(Psub_group[r[2]][j]-Psub_group[r[3]][j])]
                                    )
                Ui=torch.cat((Ui,cat),dim=0)
            else: Ui=torch.cat((Ui,torch.Tensor([Psub_group[i][j]])),dim=0)
        return Ui
    
    def rand_2_bin(self,i,Psub_group):

        jrand=random.randint(0,Psub_group[0].size(0))
        Ui=torch.Tensor()
        for j in range(Psub_group[0].size(0)):
            if random.uniform(0,1)<self.CR[i][j] or j==jrand:
                r=[] 
                r_toPick=list(range(NP))
                for pickr in range(5):
                    temp=random.choice(r_toPick)
                    r.append(temp)
                    r_toPick.remove(temp)
                cat=torch.Tensor(
                    [Psub_group[r[0]][j]+self.F[i]*(Psub_group[r[1]][j]-Psub_group[r[2]][j])
                                        +self.F[i]*(Psub_group[r[3]][j]-Psub_group[r[4]][j])]
                    )
                Ui=torch.cat((Ui,cat),dim=0)
            else: Ui=torch.cat((Ui,torch.Tensor([Psub_group[i][j]])),dim=0)
        return Ui
    
    def current_to_rand_1(self,i,Psub_group):
        r=[] 
        r_toPick=list(range(NP))
        for pickr in range(3):
            temp=random.choice(r_toPick)
            r.append(temp)
            r_toPick.remove(temp)
        Ui=Psub_group[i]+self.K*(Psub_group[r[0]]-Psub_group[r[1]])\
                        +self.F[i]*(Psub_group[r[1]]-Psub_group[r[2]])
        return Ui


In [23]:
def SaDE_Handler(G,Psub_group,index,img,label_onehot):
    Fs=[]
    strategy=Strategy_Candidate(Psub_group)
    Group_Evaluate(Fs,index,Psub_group,img,label_onehot)
    
    U=[]
    for i in range(NP):
        U.append(strategy.Action(i,Psub_group,Fs))
        
    Fsu=[]
    Group_Evaluate(Fsu,index,U,img,label_onehot)
    
    Psub_group_new=[]
    
    ns=0
    nf=0
    best=Fs.index(max(Fs))
    Psub_best=Psub_group[best]
    Fs_best=Fs[best]
    for i in range(NP):
        if Fsu[i]>Fs[i]:
            Psub_group_new[i]=U[i]
            Fs[i]=Fsu[i]
            ns+=1
            if Fsu[i]>Fs_best:
                Psub_best=U[i]
                Fs_best=Fsu[i]
        else:
            Psub_group_new[i]=Psub_group[i]
            nf+=1
    
    print("Finished operation with the size of each element in U:",U[len(U)-1].size())

In [18]:
def CC_Handler(Dindices):
    #ND法分解参数
    Pcc=[]
    Neural_Decompose(Pcc)
    #求解成熟度，储存在sub
    sub_maturity=Get_Maturity(Pcc,Dindices)
    sub_rank_index = Maturity_Rank(sub_maturity)
    #sub_rank_index=sub_rank_index[:Q]
    task=0
    for G in range(maxEAGen):#主循环
        print("Executing Evolution Generation",G,":")
        task=0
        while task<Q:
            Ds_Sampler=Ds_Sample(Dindices)
            Ds_Loader=DataLoader(dataset=train_dataset,sampler=Ds_Sampler,batch_size=szds)
            for img,label in Ds_Loader:#每个szds的batch解决一个task
                label_onehot=torch.zeros(szds,10).scatter(1,label.reshape(szds,1),1)
                img=img.view(szds, -1)
                
                index=sub_rank_index[task]
                
                Psub_group=Spawn_Group(Pcc[index])
                Fs=[]
                Group_Evaluate(Fs,index,Psub_group,img,label_onehot)
                SaDE_Handler(G,Psub_group,index,img,label_onehot)

                
                
                task+=1
                if task>=Q: break
        

In [19]:
def Batch_BP(img,label,criterion,optimizer):
    img=img.view(miniBatch, -1)
    label_onehot=torch.zeros(miniBatch,10).scatter(1,label.reshape(miniBatch,1),1)
    out=net(img)

    #loss=criterion(out,label)
    loss=criterion(out,label_onehot)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data

In [20]:
def D_Sample():
    indices = list(range(len(train_dataset)))
    random.shuffle(indices)
    return indices[:szd],sampler.SubsetRandomSampler(indices[:szd])

In [21]:
train_loss_pre=1
for epoch in range(1,maxBPEpoch+1):
    
    Dindices,train_sampler=D_Sample()
    train_loader=DataLoader(dataset=train_dataset,sampler=train_sampler,batch_size=miniBatch)

    train_loss_now=0
    for img,label in train_loader:
        train_loss_now+=Batch_BP(img,label,criterion,optimizer)

    train_loss_now/=len(train_loader)
    print('BP epoch: {}, Train Loss: {:.6f}'.format(epoch, train_loss_now))

    #if(train_loss_pre-train_loss_now<tao/math.sqrt(epoch)):
    CC_Handler(Dindices)
    train_loss_pre=train_loss_now

BP epoch: 1, Train Loss: 0.126317
Getting Maturity of all subtasks, listed below:
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 1
Executing Evolution Generation 0 :
Individuals of Subtask 9 Evaluated Fs
rand_to_best_2_bin executing on individual 0
rand_to_best_2_bin executing on individual 1
rand_1_bin executing on individual 2
rand_to_best_2_bin executing on individual 3
rand_1_bin executing on individual 4
rand_to_best_2_bin executing on individual 5
rand_to_best_2_bin executing on individual 6
rand_1_bin executing on individual 7
rand_2_bin executing on individual 8
rand_2_bin executing on individual 9
rand_to_best_2_bin executing on individual 10
rand_to_best_2_bin executing on individual 11
rand_to_best_2_bin executing on individual 12
rand_to_best_2_bin executing on individual 13
rand_1_bin executing on individual 14
rand_2_bin executing on individual 15
rand_to_best_2_bin executing on individual 16
rand_2_bin executing on individual 17
current_to_rand_1 executing on individual 18
curren

In [ ]:
out=net(train_dataset[59999][0].view(1,-1))
print(out)

In [ ]:
plt.imshow(train_dataset[59999][0].squeeze(),cmap="gray")
train_dataset[59999][0].size()